# Wpowadzenie do deep learning w bibliotece Flux.jl

## Wstęp

- [Flux](http://fluxml.ai/) jest biblioteką Julii przeznaczoną do tworzenia modeli uczenia maszynowego.
- Jest w całości oparta na Julii, przez co trywialne jest jej modyfikowanie i dostosowywanie do swoich potrzeb. 
- Możliwe jest przy tym wykorzystanie wewnątrz modeli składni, funkcji i makr Julii.
- Przy czym tworzenie całkiem złożonych standardowych modeli jest intuicyjne i szybkie, zazwyczaj zajmują one jedynie kilka linijek.

## Przykład

Na dzisiejszych zajęciach skupimy się na przykładzie sieci MLP służącej do rozpoznawania odręcznie pisanych cyfr w zbiorze MNIST. Kod źródłowy dostępny jest [tutaj](https://github.com/FluxML/model-zoo/tree/master/vision/mnist).



### Implementacja

In [1]:
using Flux, Flux.Data.MNIST, Statistics, LinearAlgebra, Images
using Flux: onehotbatch, onecold, crossentropy, throttle, Tracker
using Base.Iterators: repeated
# using CuArrays

Pierwszym krokiem jest oczywiście odpowiednie przygotowanie danych na których będziemy pracowali:

In [ ]:
# Classify MNIST digits with a simple multi-layer-perceptron

imgs = MNIST.images();

In [ ]:
Gray.(imgs[1])

In [ ]:
# Stack images into one large batch
X = hcat(float.(reshape.(imgs, :))...) |> gpu;

In [ ]:
labels = MNIST.labels()
# One-hot-encode the labels
Y = onehotbatch(labels, 0:9) |> gpu;

i zbiór testowy:

In [ ]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu;
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu;

####  Definiowanie modelu

Gdy dane są już gotowe kolejnym krokiem jest odpowiednie zdefiniowanie modelu na którym będziemy pracować. 

Zacznijmy od ręcznego zdefiniowania prostej warstwy sieci z sigmoidalną funkcją aktywacji:

In [ ]:
W = rand(4, 8)
b = rand(4)

In [ ]:
layer₁(x) = 1.0 ./ (1.0.+exp.(-W*x - b))

In [ ]:
x = rand(8)
layer₁(x)

Gdybyśmy chcieli wyuczyć ten model we Fluxie to powyższa definicja regresji logistycznej nam wystarczy - musimy jedynie zadeklerować <tt>W</tt> i <tt>b</tt> jako trenowalne parametry:

In [ ]:
using Flux.Tracker

W = param(W)
b = param(b)

Oczywiście pracując na Fluxie nie musimy deklarować wszystkiego ręcznie, dostarcza on najpopularniejsze [funkcje aktywacji](http://fluxml.ai/Flux.jl/stable/models/layers.html#Activation-Functions-1), które możemy wykorzystać w naszym modelu:

In [ ]:
layer₂(x) = σ.(W * x .+ b)

In [ ]:
layer₂(x)

Analogicznie nie ma konieczności definiowania [warstw modelu](http://fluxml.ai/Flux.jl/stable/models/layers.html#Basic-Layers-1) ręcznie:

In [ ]:
layer₃ = Dense(8,4,σ)

In [ ]:
layer₃(x)

Przy czym gdy żadna z dostarczonych razem z Fluxem definicji warstwy nam nie odpowiada możemy w banalny sposób zadeklarować własną:

In [ ]:
struct Affine
  W
  b
end

Affine(in::Integer, out::Integer) =
  Affine(param(randn(out, in)), param(randn(out)))

# Overload call, so the object can be used as a function
(m::Affine)(x) = m.W * x .+ m.b

a = Affine(10, 5)

a(rand(10)) # => 5-element vector

Gdy chcemy móc w pełni wykorzystać wszystkie przydatne funkcje Fluxa musimy jeszcze skorzystać z makra <tt>treelike</tt>:

In [ ]:
Flux.@treelike Affine

Chcąc zbudować model z więcej niż jedną warstwą musimy go odpowiednio zdefiniować.

In [ ]:
Layer₁ = Dense(28^2, 32, relu)
Layer₂ = Dense(32, 10)
Layer₃ = softmax

In [ ]:
m₁ = Chain(Layer₁ , Layer₂, Layer₃) |> gpu

Funkcja <tt>Chain</tt> pozwala łączyć w łancuchy dowolne funkcje w Julii:

In [ ]:
chain = Chain(x -> x^2, x-> -x);

In [ ]:
chain(5)

albo też:

In [ ]:
m₂(x) = Layer₃(Layer₂(Layer₁(x)))

lub jako złożenie funkcji:

In [ ]:
m₃(x) = Layer₁ ∘ Layer₂ ∘ Layer₃  

czy też:

In [ ]:
m₄(x) = Layer₁(x) |> Layer₂  |> Layer₃ 

Mając już gotową definicję modelu możemy przejść do kolejnego punktu - wyboru funkcji celu i regularyzacji modelu.

#### Funkcja straty, regularyzacja

Tak jak poprzednio funkcję straty możemy zdefiniować samodzielnie:

In [ ]:
model = Dense(5,2)

x, y = rand(5), rand(2);

In [ ]:
loss(ŷ, y) = sum((ŷ.- y).^2)/ length(y)

In [ ]:
loss(model(x), y) 

Wykorzystać [jedną z zaimplementowanych we Fluxie:](https://github.com/FluxML/Flux.jl/blob/8f73dc6e148eedd11463571a0a8215fd87e7e05b/src/layers/stateless.jl)

In [ ]:
Flux.mse(model(x),y)

Podobnie [regularyzacja](http://fluxml.ai/Flux.jl/stable/models/regularisation.html) jest dość [intuicyjna](http://fluxml.ai/Flux.jl/stable/models/layers.html#Normalisation-and-Regularisation-1):

In [ ]:
penalty() =  norm(model.W) + norm(model.b)
loss(ŷ,y) = Flux.mse(ŷ,y) + penalty()

In [ ]:
loss(model(x),y)

lub nawet prościej:

In [ ]:
loss(ŷ,y) = Flux.mse(ŷ,y) + sum(norm,params(model))

In [ ]:
loss(model(x),y)

Inne metody regularyzacji mogą być [zaimplementowane jako warstwy:](http://fluxml.ai/Flux.jl/stable/models/layers.html#Normalisation-and-Regularisation-1)

In [ ]:
model = Chain(Dense(28^2, 32, relu),
    Dropout(0.1),
Dense(32, 10),
BatchNorm(64, relu),
softmax)

W budowanym modelu funkcja straty wyglądać będzie następująco:

In [ ]:
loss(x, y) = crossentropy(m₁(x), y)

#### Uczenie modelu

Po zdefiniowaniu modelu i funkcji celu możemy przystąpić do trenowania sieci.

Podstawowym elementem pozwalającym uczyć modele jest odpowiedni algorytm liczący gradienty. We Fluxie implementacja wygląda następująco:

In [ ]:
f(x) = 3x^2 + 2x + 1

# df/dx = 6x + 2
df(x) = Tracker.gradient(f, x)[1]

df(2) # 14.0 (tracked)

# d²f/dx² = 6
d2f(x) = Tracker.gradient(df, x)[1]

d2f(2) # 6.0 (tracked)

Gdy funkcja ma wiele zmiennych wagi możemy przechowywać jako kolekcję i w ten sposób przekazywać do funkcji 

In [ ]:
W = param(2) # 2.0 (tracked)
b = param(3) # 3.0 (tracked)

f(x) = W * x + b

par = Flux.Params([W, b])
grads = Tracker.gradient(() -> f(4), par)

grads[W] # 4.0
grads[b] # 1.0

Zdefiniujmy prosty gradient:

In [ ]:
function simple_grad!(ps, η = .0001)
  for w in ps
    w.data .-= w.grad .* η
    w.grad .= 0
  end
end

Możliwe jest też zdefiniowanie [własnego algorytmu liczącego gradient](http://fluxml.ai/Flux.jl/stable/internals/tracker.html#Custom-Gradients-1) i przekazanie go do modelu - wszystkie metody, które mogliśmy dotychczas wykorzystać nadal będą działaly.

Własnoręczne definiowanie procesu optymalizacji nie jest konieczne, można wykorzystać [gotowy algorytm](http://fluxml.ai/Flux.jl/stable/training/optimisers.html) zaimplementowany we Fluxie:

In [ ]:
opt = ADAM(params(m₁))

Zdefiniujmy prostą pętlę kontrolującą uczenie:

In [ ]:
i = 1
while true
  back!(loss(model(x),y))
  max(maximum(abs.(W.grad)), abs(b.grad[1])) > 0.001 || break
  update!((W, b))
  i += 1
end

Flux jest zdolny do kontrolowania całej procedury uczenia, nie musimy robić tego samodzielnie. Służy do tego funkcja <tt>train!</tt>:

In [ ]:
Flux.train!(objective, data, opt)

Warto jednak zaznaczyć, że pozwala ona na uczenie jedynie przez pojedynczą epokę. Aby móc kontynuować proces uczenia dalej musimy w odpowiedni sposób przystować dane z których korzystamy:

In [ ]:
dataset = repeated((X, Y), 200)

albo skorzystać z makra <tt>@epochs</tt>:

In [ ]:
Flux.@epochs 2 println("hello")

Pozwala ona też na definiowanie wywołań, które pozwolą nam kontrolować przebieg uczenia.



In [ ]:
evalcb = () -> @show(loss(tX, tY))

Po omówieniu wszystkich elementów składowych biblioteki możemy złożyć naszą sieć w całość:

In [ ]:
imgs = MNIST.images()
# Stack images into one large batch
X = hcat(float.(reshape.(imgs, :))...) |> gpu

labels = MNIST.labels()
# One-hot-encode the labels
Y = onehotbatch(labels, 0:9) |> gpu

# Test set 
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu


m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(accuracy(tX, tY))
opt = ADAM(params(m))

Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 10))

println("training set accuracy: $(accuracy(X, Y))")

println("test set accuracy: $(accuracy(tX, tY))")

Oczywiście taka sieć może liczyć się strasznie długo. Ten proces można przyśpieszyć za pomocą wbudowanym w Julię wsparciu [obliczeń na GPU](http://fluxml.ai/Flux.jl/stable/gpu.html).

## Alternatywy

Flux nie jest jedyną biblioteką, która umożliwia budowanie modeli uczenia maszynowego w Julii. Poniżej wymienionych jest kilka alternatywnych możliwości:

- [Knet.jl](https://github.com/denizyuret/Knet.jl)
- [MXnet.jl](https://github.com/dmlc/MXNet.jl)
- [TensorFlow.jl](https://github.com/malmaud/TensorFlow.jl)